<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/108_%E3%83%87%E3%82%A3%E3%82%B9%E3%82%AF%E3%83%AA%E3%83%97%E3%82%BF%E3%83%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ディスクリプター
================

ディスクリプター
----------------

### ディスクリプタープロトコル ###

**ディスクリプター**（descriptor）とは、**ディスクリプタープロトコル**（descriptor protocol）で定義されたメソッドを持つオブジェクトのことをいう。ディスクリプタープロトコルが定めるメソッドは以下のとおり。

  * `descr.__get__(self, instance, owner=None)`
  * `descr.__set__(self, instance, value)`
  * `descr.__delete__(self, instance)`

これらのメソッドのいずれかを定義すれば、オブジェクトはディスクリプターとみなされる。

`__set__()` と `__delete__()` の両方を定義するものを**データディスクリプター**、`__get__()` だけを定義するものを**非データディスクリプター**と呼ぶ。

データディスクリプターのうち、`__get__()` と `__set__()` の両方を定義し、`__set__()` が呼び出されたときに `AttributeError` を送出するものを、**読み出し専用データディスクリプター**と呼ぶ。

また、ディスクリプターは、次の特殊メソッドを持つことがある。

  * `descr.__set_name__(self, owner, name)`

あるクラスのクラス変数に `__set_name__()` メソッドを持つオブジェクトをアサインしている場合、そのクラス（**オーナークラス**と呼ぶ）の定義を実行すると、オーナークラスのクラスオブジェクトとアサインされたクラス属性の名前を伴って `__set_name__()` が自動的に呼び出される（この動作はオブジェクトがディスクリプターである場合に限らない）。

次のコードでは、`MyDescriptor` のインスタンスはデータディスクリプターであり、かつ、`__set_name__()` メソッドを持ち、`A` のクラス変数 `x` にアサインしている。

In [ ]:
class MyDescriptor:
    def __get__(self, instance, owner):
        pass

    def __set__(self, instance, value):
        pass

    def __delete__(self, instance):
        pass

    def __set_name__(self, owner, name):
        print(f"{owner=}, {name=}")

class A:
    x = MyDescriptor()

owner=<class '__main__.A'>, name='x'


`A` のクラス定義を実行すると、`x = MyDescriptor()` というアサインによって `x.__set_name__(A, 'x')` が自動的に呼び出される。`__set_name__()` は、オーナークラスとアサインされたクラス属性の名前を知る必要がある場合に便利である。

このようにクラス変数にディスクリプターをアサインしている場合、Python は属性アクセスのデフォルトの振る舞いをオーバーライドし、代わりにディスクリプターメソッドを呼び出す。属性の取得・設定・削除では、デフォルトでは属性辞書の項目の取得・設定・削除が行われるが、それぞれ代わりに `__get__()` ・ `__set__()` ・ `__delete__()` の呼び出しが自動的に行われるということである。ただし、このディスクリプター呼び出しが発動する条件は複雑で、オブジェクトの種類とディスクリプターの種類に依存している。

### インスタンスからの呼び出し ###

インスタンスの属性参照 `a.x` およびこれと等価な `getattr(a, 'x')` では、`__getattribute__(a, 'x')` というメソッド呼び出しが行われる。ユーザー定義クラスは、このメソッドを `object` から継承する。 `object.__getattribute__()` は、以下のロジックを定めている。

  * まず `type(a).__dict__['x']`、さらに `type(a)` のメソッド探索順序（MRO）に従ったさらなる探索でクラス変数 `x` の値を探す。
  * `x` の値がディスクリプターであるかどうかを確認する。
  * そのうえで以下の優先順位を付ける。
      1. データディスクリプター
      2. インスタンスの属性辞書（`a.__dict__['x']` を返す）
      3. 非データディスクリプター
      4. クラス変数（`x` の値をそのまま返す）
  * 上記のいずれも使用できない場合は、 `AttributeError` 例外を送出する。

さらに、ドット演算子 `.` と組み込み関数 `getattr()` には、`__getattr__()` のフックが存在する。`__getattribute__()` が `AttributeError` を発出したときに `__getattr__(a, 'x')` が呼び出される。インスタンスが `__getattr__()` メソッドを持っていないときには、`AttributeError` を再送出する。

上記の優先順位によって、データディスクリプターは必ずインスタンスの属性辞書をオーバーライドするのに対して、非データディスクリプターはインスタンスの属性辞書にオーバーライドされることがある。

`a.x` に対してディスクリプターが使用される場合、 `desc.__get__(a, type(a))` という形式で呼び出される。

以上のロジックは C で書かれているが、これと等価な Python のコードが[公式ドキュメント](https://docs.python.org/ja/3/howto/descriptor.html#invocation-from-an-instance)に載っている。

### クラスからの呼び出し ###

クラスの属性参照 `A.x` でも `__getattribute__()` メソッド呼び出しが行われるが、このメソッドは `type` から継承する。 `type.__getattribute__()` が定めるロジックは、 `object.__getattribute__()` と似ているが、インスタンスの属性辞書はクラスの MRO を通じた探索と置き換えられている。

`A.x` に対してディスクリプターが使用される場合、 `desc.__get__(None, A)` という形式で呼び出される。

### super からの呼び出し ###

組み込み関数 `super()` を伴った属性参照では、`super()` によって返されるオブジェクトが持つ `__getattribute__()` メソッドの呼び出しが行われる。`super(A, obj).x` という属性参照は、`obj.__class__.__mro__` を探索して、 `A` の直前のクラス `B` をまず探し、 `B.__dict__['x'].__get__(obj, A)` を返す。もしディスクリプターでなければ `x` を変更せずに返す。

関数とメソッド
--------------

実は、Python の関数は非データディスクリプターである。次は `__get__()` を直接呼び出す例である:

In [ ]:
def f(a, b):
    return a + b

print(f"{f = }")
print(f"{f.__get__(1) = }")
print(f"{f.__get__(1)(2) = }")

f = <function f at 0x7d64b552fa30>
f.__get__(1) = <bound method f of 1>
f.__get__(1)(2) = 3


関数が非データディスクリプターであることは、関数とメソッドをシームレスに組み合わせることに利用されている。関数クラスは C で書かれた非公開のクラスであるが、そのディスクリプタープロトコルに関する実装を Python による疑似コードで表すと、次のようになる。

``` python
class Function:
    ...

    def __get__(self, obj, objtype=None):
        "Simulate func_descr_get() in Objects/funcobject.c"
        if obj is None:
            return self
        return MethodType(self, obj)
```

関数 `f` がオブジェクト `a` からディスクリプターとして `f.__get__(a, type(a))` のように呼び出される場合は、 `MethodType` インスタンスが返される。 `MethodType` は、ユーザー定義クラスのインスタンスメソッドの型であり、実行時に関数をメソッドに変換するために使われる。

Python ではメソッドと通常の関数の違いは、オブジェクトのインスタンスが他の引数よりも前に渡されるか否かという違いでしかない。インスタンスが渡される引数は、慣習的に `self` という名前が使われる。`self` は、PEP 8 などのコーディング規約上の命名であって、予約語ではない。C++ や Java では、`this` キーワードがインスタンスを参照する特別なポインタとされ、引数に明示する必要はない。これに比べると Python の `self` はどこから来るのかわかりにくいが、`MethodType` によって追加される引数がその正体である。

`MethodType` は、クラス、クラスインスタンスと任意の関数（あるいは呼び出し可能オブジェクト）を結びつける。以下の特殊属性を持つ。

| 特殊属性 | 意味 |
|:--|:----|
| `__self__` | メソッドがバインドされているクラスインスタンス |
| `__func__` | もとの関数オブジェクト |

C による `MethodType` の実装を Python による疑似コードで表すと、次のようになる。

``` python
class MethodType:
    "Emulate PyMethod_Type in Objects/classobject.c"

    def __init__(self, func, obj):
        self.__func__ = func
        self.__self__ = obj

    def __call__(self, *args, **kwargs):
        func = self.__func__
        obj = self.__self__
        return func(obj, *args, **kwargs)
```

オブジェクト `a` のクラスのクラス変数 `m` に関数 `f` をアサインすると、`a.m` のような参照は、非データディスクリプター呼び出しが使用され、`MethodType` インスタンス（インスタンスメソッド）が返される。その `__func__` 属性の値は `f` であり、`__self__` 属性の値は `a` である。

`a.m()` のようなインスタンスメソッドの呼び出しは、呼び出し可能オブジェクトの呼び出し形式に適用されるルールにより、実行時に `type(m).__call__(m, arg1, ...)` のように暗黙的に変換される。いま `m` はインスタンスメソッド、つまり `MethodType` のインスタンスであり、その`__call__()` の中では、もとの関数 `f` を使って `f(a, arg1, ...)` が実行され、結果が返される。こうして、第 1 引数にインスタンスが渡される。

まとめると、メソッドの呼び出しは以下のように変換される。

　`a.m(arg1, ...)` → `type(a).__dict__['m'].__get__(a, type(a))(arg1, ...)` → `MethodType(f, a)(arg1, ...)`  
　 → `MethodType.__call__(MethodType(f, a), arg1, ...)` → `f(a, arg1, ...)`

次のコードでは、インスタンスメソッドの呼び出しと等価な呼び出しを確認できる:

In [ ]:
def f(self, a, b):
    return a + b

class A:
    m = f  # def m(self, a, b): ... と直接定義するのと等価

a = A()
assert a.m(1, 2) == type(a).__dict__["m"].__get__(a, type(a)).__call__(1, 2) == f(a, 1, 2)

非データディスクリプターは、インスタンスの属性辞書にオーバーライドされることに注意する。

また、`A.m()` のような呼び出しでは、`__get__()` が第 1 引数に `None` を渡されるので関数自身を返すことから、以下のように変換される。

　`A.m(arg1, ...)` → `A.__dict__['m'].__get__(None, A)(arg1, ...)` → `f(arg1, ...)`

**【メモ】**  
インスタンスメソッドの呼び出しは、`.` の解決のための探索と、メソッド化のための変換が行われるので、それなりに重い処理となる。リストの生成で `append()` メソッドを使う方法がリスト内包表記を使う方法より時間がかかる原因の 1 つはこれである。同じメソッドを頻繁に呼び出す場合は、メソッド化までの処理結果を変数に保存しておき、その変数で呼び出すことにより処理時間を短縮化することができる（メモ化に似た効果がある）。次の 2 つのコードは、普通に `append()` メソッドを使用してリストの生成にかかる時間と、メソッド化までの処理結果を変数に保存してリストの生成にかかる時間を計測している。

In [ ]:
%%timeit
a = []
for i in range(100000):
    a.append(0)

7.59 ms ± 2.2 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [ ]:
%%timeit
a = []
a_append = a.append
for i in range(100000):
    a_append(0)

4.85 ms ± 526 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


メソッド化までの処理結果を変数に保存して呼び出すほうが大幅に時間短縮となることがわかる。それでも、関数呼び出しのコストがあるので、リスト内包表記よりは時間がかかる。

クラスメソッド
--------------

C++ や Java ではクラスメソッドとスタティックメソッドは同義だが、Python ではクラスもオブジェクトであることから両者は異なる。

Python において、**クラスメソッド**とは、引数リストの先頭にクラス自身を参照する引数が加えられるメソッドのことをいう。具体的には、組み込みデコレーター `classmethod()` を適用したメソッドのことをいう。ただし、`__init_subclass__()` メソッドは特別扱いされ、`classmethod()` の適用がなくても自動的にクラスメソッドとなる。

PEP 8 は、クラスメソッドのはじめの引数の名前は `cls` を使うものとする。

第 1 引数としてクラスを取るため、クラスメソッドは、`cls.var` のようにクラス属性を参照できるほか、`cls()` のようにインスタンスを生成して返すことができる一方、インスタンス属性を参照することはできない（`cls.var` への代入はクラス属性を変更する）。

In [ ]:
class Book:
    """本のタイトルと著者を管理するクラス"""

    def __init__(self, title, author):
        self.title = title
        self.author = author

    @classmethod
    def from_dict(cls, book_dict):
        """辞書からインスタンスのリストを生成する"""
        return [cls(title, author) for title, author in book_dict.items()]

    def __repr__(self):
        return "Book({}, {})".format(self.title, self.author)


if __name__ == "__main__":
    book_dict = {"Introducing Python": "Bill Lubanovic", "Effective Python": "Brett Slatkin"}
    print(Book.from_dict(book_dict))

[Book(Introducing Python, Bill Lubanovic), Book(Effective Python, Brett Slatkin)]


実のところ、`classmethod` は、非データディスクリプターである。そのディスクリプタープロトコルに関する実装を Python による疑似コードで表すと、次のようになる。

``` python
import functools

class ClassMethod:
    "Emulate PyClassMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f
        functools.update_wrapper(self, f)

    def __get__(self, obj, cls=None):
        if cls is None:
            cls = type(obj)
        return MethodType(self.f, cls)
```

コンストラクタ引数 `f` は、デコレーターでラップされる関数であり、これはインスタンスメソッド化される前の関数である。 `__get__()` メソッドは、一般の関数クラスと違って、`obj` 引数が `None` の場合に関数自身を返すのではなく、`MethodType(f, cls)` で生成されるメソッドを返す。このため、クラスメソッドの場合、`C.f` のようなクラスからの呼び出しでは、以下のように変換される。

　`C.f(arg1, ...)` → `C.__dict__['f'].__get__(None, C)(arg1, ...)` → `MethodType(f, C)(arg1, ...)`   
　 → `MethodType.__call__(MethodType(f, C), arg1, ...)` → `f(C, arg1, ...)`

こうして、第 1 引数 `cls` にクラス自身への参照が渡されるのである。

一方、`c.f` のようなインスタンスからの呼び出しでは

　`c.f(arg1, ...)` → `C.__dict__['f'].__get__(c, C)(arg1, ...)` → `MethodType(f, C)(arg1, ...)`  
　 → `MethodType.__call__(MethodType(f, C), arg1, ...)` → `f(C, arg1, ...)`

となって、クラスからの呼び出しと同じ結果となる。

In [ ]:
class C:
    @classmethod
    def f(cls, x):
        return cls.__name__, x

assert C.f(3) == C().f(3) == ('C', 3)

スタティックメソッド
--------------------

**スタティックメソッド**（または**静的メソッド**）は、インスタンス化せずに使えるメソッドであり、暗黙の第一引数を受け取らない。具体的には、組み込みデコレーター `staticmethod()` を適用したメソッドのことをいう。

In [ ]:
class C:
    @staticmethod
    def f(x):
        return x * 10

assert C.f(3) == C().f(3)

`__new__()` メソッドは特別扱いされ、`staticmethod()` の適用がなくても自動的にスタティックメソッドとなる。

関数とメソッドの違いは暗黙の第一引数を受け取るか否かであったから、スタティックメソッドは実質的には普通の関数と同じである。スタティックメソッドの中では、インスタンス属性もクラス属性も参照することはできない。

`staticmethod` 型の Python による疑似コードで表すと、次のようになる。

``` python
import functools

class StaticMethod:
    "Emulate PyStaticMethod_Type() in Objects/funcobject.c"

    def __init__(self, f):
        self.f = f
        functools.update_wrapper(self, f)

    def __get__(self, obj, objtype=None):
        return self.f

    def __call__(self, *args, **kwds):
        return self.f(*args, **kwds)
```

非データディスクリプターとしては自身を返すだけである。

通常は、普通の関数として書ける処理をクラスのスタティックメソッドにするメリットはないであろう。たとえば、特定のクラスのインスタンスに対して処理を行う関数は、そのクラスにスタティックメソッドとして持たせると関連性がわかりやすいかもしれない。

プロパティ
----------

インスタンスのデータ属性のようにアクセスすると関数の呼び出しを引き起こすものを**プロパティ**と呼ぶ。具体的には、組み込みのデータディスクリプタークラス `property` のインスタンスである。

``` python
x = property(fget=None, fset=None, fdel=None, doc=None)
```

`fget`、`fset`、`fdel` は、ふつう、すべて同じデータ属性（本当の属性）を扱うメソッドである。これらに扱われるデータ属性をプロパティの属性値と呼ぶ。`doc` は、プロパティの docstring である。

`c` がプロパティ `x` を持つクラスのインスタンスならば

  * `c.x` は `fget()` を呼び出す。`fget()` はふつう属性値を返し、**ゲッター**（getter）と呼ばれる。
  * `c.x = value` は `fset(value)` を呼び出す。`fset()` はふつう属性値を設定し、**セッター**（setter）と呼ばれる。
  * `del c.x` は `fdel()` を呼び出す。`fdel()` はふつう属性を削除し、**デリーター**（deleter）と呼ばれる。

属性アクセスについて、データディスクリプターがインスタンスの属性辞書より優先されるので、プロパティが定義されるときには、`c.x = value` は、つねに、プロパティへのアクセスとされる。

In [ ]:
class C:
    def __init__(self):
        self.__x = None  # プロパティの属性値（サブクラスで使用しないために難号化）

    def getx(self):
        return self.__x

    def setx(self, value):
        print("in setx")
        self.__x = value

    def delx(self):
        print("in delx")
        del self.__x

    x = property(getx, setx, delx, "I'm the 'x' property.")

c = C()
assert c.x is None
c.x = "hoge"
del c.x

in setx
in delx


プロパティは、属性値を取得・設定・削除する際に何らかの処理をしたいときに便利である。たとえば、属性値に特定の型を要求するとき、`fset()` の中で引数 `value` の型をチェックすることができる。また、`fset` と `fdel` を設定しないとき、プロパティは読み取り専用となる。

クラス `property` の実装において、データディスクリプタープロトコルのメソッド以外のメソッドも定義されている。これを Python による疑似コードで表すと、次のようになる。

``` python
class Property:
    "Emulate PyProperty_Type() in Objects/descrobject.c"

    def __init__(self, fget=None, fset=None, fdel=None, doc=None):
        self.fget = fget
        self.fset = fset
        self.fdel = fdel
        if doc is None and fget is not None:
            doc = fget.__doc__
        self.__doc__ = doc
        self._name = ''

    def __set_name__(self, owner, name):
        self._name = name

    def __get__(self, obj, objtype=None):
        if obj is None:
            return self
        if self.fget is None:
            raise AttributeError(f"property '{self._name}' has no getter")
        return self.fget(obj)

    def __set__(self, obj, value):
        if self.fset is None:
            raise AttributeError(f"property '{self._name}' has no setter")
        self.fset(obj, value)

    def __delete__(self, obj):
        if self.fdel is None:
            raise AttributeError(f"property '{self._name}' has no deleter")
        self.fdel(obj)

    def getter(self, fget):
        prop = type(self)(fget, self.fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def setter(self, fset):
        prop = type(self)(self.fget, fset, self.fdel, self.__doc__)
        prop._name = self._name
        return prop

    def deleter(self, fdel):
        prop = type(self)(self.fget, self.fset, fdel, self.__doc__)
        prop._name = self._name
        return prop
```

`getter()`、`setter()`、`deleter()` はデコレーターであり、それぞれ独立にデータディスクリプターを生成する。それぞれがプロパティを束縛するときに、自動的に `__set_name__()` が共通のプロパティの名前を `self._name` に格納し、その値が各ディスクリプターの属性 `_name` に設定される（`AttributeError` 処理時のメッセージに利用される）。`property()` も第1引数の `fget` に関してデコレーターとなっているから、`getter()` と同等の働きをする。したがって、@ 構文を使って、次のようにプロパティを定義することができる。

In [ ]:
class C:
    def __init__(self):
        self.__x = None

    @property
    def x(self):
        """I'm the 'x' property."""
        return self.__x

    @x.setter
    def x(self, value):
        self.__x = value

    @x.deleter
    def x(self):
        del self.__x

&#x5f;&#x5f;slots&#x5f;&#x5f;
---------

クラス変数 `__slots__` に、インスタンスが用いる変数名の文字列、またはタプルかリストを代入すると、各インスタンスに対して宣言された変数に必要な記憶領域を確保し、`__dict__` と `__weakref__` が自動的に生成されなくなる。`__dict__` を使うのに比べて、メモリ消費量を節約でき、また、属性探索を高速化できる。反面、`__slots__` に列挙されていない新たな変数をインスタンスに追加することはできない。

次のコードは、`sys.getsizeof()` 関数を使用して、`__slots__` を使用するクラスオブジェクトと使用しないクラスオブジェクトのメモリ消費量を比較する例である。

In [ ]:
class C:
    __slots__ = ["x", "y"]

    def __init__(self, x):
        self.x = x

c = C(1)
assert not hasattr(c, "__dict__")  # __dict__ 属性を持たない
c.y = 2  # `__slots__` に列挙される名前は、インスタンスに属性として追加できる
try:
    c.name = "hoge"
except AttributeError:
    print("クラス変数 __slots__ に列挙されていない新たな名前の属性を追加することはできない")

# 普通のクラスとメモリ消費量を比較する
class D:
    def __init__(self, x):
        self.x = x

import sys
print(f"{sys.getsizeof(C)=}")
print(f"{sys.getsizeof(D)=}")

クラス変数 __slots__ に列挙されていない新たな名前の属性を追加することはできない
sys.getsizeof(C)=904
sys.getsizeof(D)=1072


`__slots__` は、クラスのレベルで各変数に対するディスクリプターを使って実装されている。[公式ドキュメント](https://docs.python.org/ja/3/howto/descriptor.html#member-objects-and-slots)には、これを Python による疑似コードで表す例が掲載されている。

型変換を行うディスクリプター
----------------------------

@tag1216 さんの [Qiita 記事](https://qiita.com/tag1216/items/5143057f2e6fe1db5abf)より、値を設定する際に指定の型に変換する非データディスクリプター `TypedField` が次のように実装される。

In [ ]:
class TypedField:

    def __init__(self, field_type):
        self.field_type = field_type

    def __set_name__(self, owner, name):
        self._name = name

    def __set__(self, obj, value):
        if not isinstance(value, self.field_type):
            value = self.field_type(value)
        obj.__dict__[self._name] = value


class MyClass:
    str_value = TypedField(str)
    int_value = TypedField(int)


if __name__ == "__main__":
    m = MyClass()
    m.str_value = 123
    m.int_value = "123"
    assert isinstance(m.str_value, str)
    assert isinstance(m.int_value, int)

非データディスクリプターの優先順位がインスタンスの属性辞書より低いので、`__get__()` を実装する必要はないことに注意する（属性辞書によりアクセスされる属性の値が取得される）。なお、当然だが、型変換ができないときには `ValueError ` が発生する。

プロパティを使って等価なコードを書こうとすると、属性ごとにゲッターとセッターを実装しなければならない。

In [ ]:
def convert(value, value_type):
    if not isinstance(value, value_type):
        value = value_type(value)
    return value


class MyClass:

    @property
    def str_value(self):
        return self.__dict__['str_value']

    @str_value.setter
    def str_value(self, value):
        value = convert(value, int)
        self.__dict__['str_value'] = value

    @property
    def int_value(self):
        return self.__dict__['int_value']

    @int_value.setter
    def int_value(self, value):
        value = convert(value, int)
        self.__dict__['int_value'] = value

部分適用
--------

**部分適用**（partial application）とは、複数の引数をとる関数の一部の引数に実引数を適用させた形の関数を作成することをいう。部分適用により作成された関数を、元の関数の**部分適用関数**と呼ぶ。

たとえば、`functools` モジュールの関数 `update_wrapper(wrapper, wrapped)` を考えると、第 2 引数に具体的な wrapped 関数 `mywrapped()` を与えた形 `update_wrapper(wrapper, mywrapped)` と等価な関数が作成されれば、その新しい関数は `wrapper` を引数に取る関数ということになる。実は、その関数こそ `functools.wraps()` デコレーターが作成する関数である。

`functools` モジュールが提供する `partial` オブジェクトは、部分適用関数のように振る舞う。`wraps(wrapped)` は、`partial(update_wrapper, wrapped=wrapped)` とほぼ等価である。

`functools.partial` のコンストラクタは次のとおり。

``` python
functools.partial(func, /, *args, **keywords)
```

第 1 引数 `func` として呼び出し可能オブジェクトを受け取り、第 2 引数以下に、`func` の引数に適用する実引数を受け取る。このインスタンス化は、もとの関数の位置引数・キーワード引数の一部を「凍結」した部分適用として使われ、簡素化された引数形式を持った新たな呼び出し可能オブジェクトを生成する。**位置引数は左から適用される**ことに注意する。

In [ ]:
from functools import partial

def point(x, y, scale=1.0):
    return x * scale, y * scale

# point の引数 x を 0.0 に固定し、引数 scale を 10.0 に固定した呼び出し可能オブジェクトを作成
slide = partial(point, 0.0, scale=10.0)
assert slide(1.0) == (0.0, 10.0)
assert slide(2.0) == (0.0, 20.0)

`functools` モジュールが提供する `partialmethod` オブジェクトは、`functools.partial` オブジェクトと似た動作をする非データディスクリプターである。コンストラクタは次のとおり。

``` python
functools.partialmethod(func, /, *args, **keywords)
```

生成される `partialmethod` オブジェクトの `__get__()` メソッドは、もとの `func` と、`args`、`keywords` から `partial` オブジェクトを生成して返す。これにより一部引数を指定したメソッドを定義することができる。

In [ ]:
from functools import partialmethod


class Point:
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def move(self, incremental, scale):
        x = self.x + incremental
        y = self.y + incremental
        self.x = x * scale
        self.y = y * scale

    scale = partialmethod(move, 0.0)


if __name__ == "__main__":
    point = Point(2.0, 3.0)
    point.scale(10.0)
    assert point.x == 20.0
    assert point.y == 30.0